## Notes
1. np.matlib is used where it was hard to find a Python translate on the fly due to the long and not-readable original Matlab code
2. There are several spots in the Matlab code that it uses **(:)** in order to linearize a matrix. There are two ways to perfom this in Python: By using numpy's **reshape()** function (meanD.shape[0]\*meanD.shape[1], 1) OR by using numpy's **flatten()** function. Both are used but flatten is a better a solution becaue of readability. Here is a one major difference though: **x1 = img1.flatten()** is of shape **(N,)** but **x2 = np.reshape(img1, (img1.shape[0]\*img1.shape[1],1))** is of shape **(N,1)**, so I think it would be safer to use the np.reshape(). However, the results for np.divide(np.sum(np.abs(**x1**)), 400\*0.5) is the same as np.divide(np.sum(np.abs(**x2**)), 400\*0.5).
3. Indexing in Python starts from zero while in Matlab starts from one, so for example mean(D,3) in Matlab is the same as np.mean(D, axis = 2)
4. mean(D) in Matlab is the same as np.mean(D,0) in Python, which is mean across columns (5 x 3) -> 1 x 3
5. To Do: Write a parse config function
6. Python indexing different from Matlab indexing, so find() in Matlab can not be used in Python
7. np.tile() is the equivalent of Matlab's repmat

In [1]:
import scipy.misc
import numpy as np
from scipy.fftpack import fft, dct, idct
import skimage.transform
from numpy import linalg as LA
from PIL import Image
import sys
import os
import matplotlib.pyplot as plt
import math

In [2]:
# To Do: write function for parsing options from a .yaml file
options = {
    'working_size' : 4 , # change based on your input data
    'darkfield' : True ,
    'basefluo' : False ,
    'lambda' : 0.0001 ,
    'lambda_dark' : 0.0001 ,
    'lambda_darkfield' : 0.0001,
    'optimization_tol' : 0.001 ,
    'max_iterations' : 500,
    'estimation_mode' : 'l0',
    'reweight_tol' : 0.004 , # please change
    'max_reweightiterations' : 50 , # please change
    'epsilon' : 0.001 # please change - also do thet meant epsilon?? 
    
} 

In [43]:
def inexact_alm_rspca_l1(D, weight, tol, maxIter, darkfield_flag, options):
    
    # weight: np.ones OR one
    # darkfield_flag: true OR false
    # B1_uplimit : darkfieldlimit OR 1e7
    
    # intialization and given default variables  
    p = D.shape[0]
    q = D.shape[1]
    m = p*q
    n = D.shape[2]
    
    D = D.reshape(m,n, order='F')
    # Todo: if weight is empty, u, s, vh = np.linalg.svd(D, full_matrices=False)set to one
    weight = weight.reshape(D.shape)
    
    u, s, vh = np.linalg.svd(D, full_matrices=False)
    norm_two = s[0]
    # Todo:clear u,s,vh
    
    Y1 = 0
    Y2 = 0
    ent1 = 1
    ent2 = 10

    A1_hat = np.zeros(D.shape)
    E1_hat = np.zeros(D.shape)
    #W_hat = mirt_dct2(mean(reshape(A1_hat,p,q,n),3));
    W_hat = dct(dct(np.mean(A1_hat.reshape(p,q,n), 2), norm='ortho'),axis=-2,norm='ortho')
    mu = 12.5/norm_two# this one can be tuned
    mu_bar = mu * 1e7
    rho = 1.5          # this one can be tuned
    d_norm = np.linalg.norm(D, ord='fro')
    A1_coeff = np.ones((1,n), dtype=float)
    A_offset = np.zeros((m,1), dtype=float)
    B1_uplimit = np.min(D)
    B1_offset = 0
    #A_uplimit = min(D,[],2)
    A_uplimit = np.min(D, 1) # Todo: This gives horizontal vector, but matlab one gives vertical
    A_uplimit = A_uplimit.reshape(A_uplimit.size, 1)
    A_inmask = np.zeros((p,q), dtype=float)
    A_inmask[int(np.round(p/6)):int(np.round(p*5/6)),int(np.round(q/6)):int(np.round(q*5/6))] = 1
    # main iteration loop starts
    itr = 0
    total_svd = 0
    converged = False
    print('Starting while loop...')
    while (not converged):
        itr = itr + 1 
        W_hat = W_hat.transpose()
        W_idct_hat = idct(idct(W_hat, norm='ortho'),axis=-2,norm='ortho') # 
        A1_hat = (W_idct_hat.reshape(W_idct_hat.size,1) * A1_coeff) + A_offset
        temp_W = np.divide((D - A1_hat - E1_hat + (1/mu)*Y1 ),ent1) #  Todo: np.substract?
        temp_W = temp_W.reshape(p,q,n)
        temp_W = np.mean(temp_W,2)
        W_hat = W_hat + dct(dct(temp_W, norm='ortho'),axis=-2,norm='ortho') # Todo: is W_hat a row vector?
        W_hat = np.maximum(W_hat - (options['lambda']/(ent1*mu)),0) + np.minimum(W_hat + (options['lambda']/(ent1*mu)),0)
        W_idct_hat = idct(idct(W_hat, norm='ortho'),axis=-2,norm='ortho')
        A1_hat = (W_idct_hat.reshape(W_idct_hat.size,1) * A1_coeff) + A_offset
        E1_hat = E1_hat + D - A1_hat - E1_hat + (1/mu)*Y1/ent1
        E1_hat = np.maximum(E1_hat - weight/(ent1*mu), 0)+ np.minimum(E1_hat + weight/(ent1*mu), 0)
        R1 = D - E1_hat
        A1_coeff = np.divide(np.mean(R1,0),np.mean(R1))
        A1_coeff = A1_coeff.reshape(1, A1_coeff.size)
        A1_coeff[A1_coeff<0] = 0
        if darkfield_flag == 1: 
            #validA1coeff_idx = find(A1_coeff<1);
            #B1_coeff = (mean(R1(W_idct_hat(:)>mean(W_idct_hat(:))-1e-6,validA1coeff_idx))-mean(R1(W_idct_hat(:)<mean(W_idct_hat(:))+1e-6,validA1coeff_idx)))./mean(R1(:));
            validA1coeff_idx = np.argwhere(A1_coeff<1)
            idx = validA1coeff_idx[:,1]
            if (validA1coeff_idx.size != 0):
                temp1 = W_idct_hat.reshape(W_idct_hat.size,1)>np.mean(W_idct_hat)-1e-6
                temp_mat_1 = R1[temp1[:,0],idx[0]]
                for i in range(1,idx.size):
                    temp_mat_1 = np.hstack((temp_mat_1, R1[temp1[:,0],idx[i]]))
                #temp_mat_1 = np.reshape(temp_mat_1, (int(temp_mat_1.shape[0]/idx.size), idx.size))
            
                temp2 = W_idct_hat.reshape(W_idct_hat.size,1)<np.mean(W_idct_hat)+1e-6
                temp_mat_2 = R1[temp2[:,0],idx[0]]
                for i in range(1,idx.size):
                    temp_mat_2 = np.hstack((temp_mat_2, R1[temp2[:,0],idx[i]]))
                #temp_mat_2 = np.reshape(temp_mat_2, (int(temp_mat_2.shape[0]/idx.size), idx.size))
                B1_coeff = np.divide((np.mean(temp_mat_1,0)-np.mean(temp_mat_2,0)),np.mean(R1))
                #B1_coeff = (mean(R1(W_idct_hat(:)>mean(W_idct_hat(:))-1e-6,validA1coeff_idx[:,0]))-mean(R1(W_idct_hat(:)<mean(W_idct_hat(:))+1e-6,validA1coeff_idx[:,0])))./mean(R1(:));
                
                k = np.size(idx)   
                temp1 = np.sum(A1_coeff[0,idx]**2)
                temp2 = np.sum(A1_coeff[0,idx]) 
                temp3 = np.sum(B1_coeff)
                temp4 = np.sum(A1_coeff[0,idx]*B1_coeff)
                temp5 = temp2 * temp3 - (k*temp4)
                if temp5 == 0:
                    B1_offset = 0
                else:
                    B1_offset = np.divide((temp1 * temp3 - temp2 * temp4), temp5)
               
               
                B1_offset = np.maximum(B1_offset,0)
                B1_offset = np.minimum(B1_offset, np.divide(B1_uplimit, np.mean(W_idct_hat)))
                B_offset = np.multiply(B1_offset, np.mean(W_idct_hat)) - np.multiply(B1_offset, W_idct_hat.reshape(W_idct_hat.size,1))
                temp_mat = R1[:,idx[0]]
                temp_mat = temp_mat.reshape(temp_mat.size,1)
                for i in range(1,idx.size):
                    temp_R = R1[:,idx[i]]
                    temp_R = temp_R.reshape(temp_R.size,1)
                    temp_mat = np.hstack((temp_mat, temp_R))
                
                temp = np.mean(temp_mat,1) - np.mean(A1_coeff[0,idx])
                A1_offset = np.multiply(temp.reshape(temp.size, 1), W_idct_hat.reshape(W_idct_hat.size, 1))
                #A1_offset = mean(R1(:,validA1coeff_idx[:,0]),2)-mean(A1_coeff(validA1coeff_idx[:,0])).*W_idct_hat(:);
                A1_offset = np.subtract(A1_offset, np.mean(A1_offset))
                A_offset = np.subtract(A1_offset, np.mean(A1_offset)) - B_offset
           
                W_offset = dct(dct(A_offset.reshape(p,q), norm='ortho'),axis=-2,norm='ortho')
                W_offset =  np.maximum(np.subtract(W_offset, options['lambda_darkfield']/ent2*mu),0) + np.minimum(np.add(W_offset, options['lambda_darkfield']/ent2*mu),0)
                A_offset= idct(idct(W_offset, norm='ortho'),axis=-2,norm='ortho')
                A_offset = A_offset.reshape(A_offset.size,1)
           
                A_offset = np.maximum(A_offset - options['lambda_darkfield']/(ent2*mu), 0.0) + np.minimum(A_offset + options['lambda_darkfield']/(ent2*mu), 0.0)          
                A_offset = A_offset + B_offset
            else:
                W_offset = np.zeros((p,q), dtype=float)
                B_offset = np.zeros((p*q,1), dtype=float)
                A_offset= idct(idct(W_offset, norm='ortho'),axis=-2,norm='ortho')
                A_offset = A_offset.reshape(A_offset.size,1)
           
                A_offset = np.maximum(A_offset - options['lambda_darkfield']/(ent2*mu), 0.0) + np.minimum(A_offset + options['lambda_darkfield']/(ent2*mu), 0.0)          
                A_offset = A_offset + B_offset
            
            
                                                                                 
        Z1 = D - A1_hat - E1_hat
        Y1 = Y1 + np.multiply(mu,Z1)
        mu = np.minimum(mu*rho, mu_bar)
        
        # stop Criterion  
        stopCriterion = np.linalg.norm(Z1, ord='fro') / d_norm
    
        if stopCriterion < tol:
            converged = True
            #A_offset = A_offset + B1_offset * W_idct_hat.reshape(W_idct_hat.size,1)
            #return A1_hat, E1_hat, A_offset
        
        if np.mod( total_svd, 10) == 0:
            print('Iteration ' , itr, ' |W|_0 ' , np.sum(np.abs(W_hat.reshape(W_hat.size,1))>0),
            ' |E1|_0 ', np.sum((np.abs(E1_hat.reshape(E1_hat.size,1)>0))),
            ' stopCriterion ', stopCriterion,
            ' B1_offset ', B1_offset)
            #A_offset = A_offset + B1_offset * W_idct_hat.reshape(W_idct_hat.size,1)
            #return A1_hat, E1_hat, A_offset
                                                                                 
        if not converged and itr >= maxIter:
            print('Maximum iterations reached') 
            converged = True
            #A_offset = A_offset + B1_offset * W_idct_hat.reshape(W_idct_hat.size,1)
            #return A1_hat, E1_hat, A_offset
    
    A_offset = A_offset + B1_offset * W_idct_hat.reshape(W_idct_hat.size,1)
    return A1_hat, E1_hat, A_offset

In [41]:
def BaSiC(IF, options):
    
     # set default values for options that are not specified
    if options['estimation_mode'] is None:
        options['estimation_mode'] = 'l0'

    if options['max_iterations'] is None:
        options['max_iterations'] = 500
        
    if options['optimization_tol']:
        options['optimization_tol'] = 1e-6

    if options['darkfield'] is None:
        options['darkfield'] = 'false'

    nrows = options['working_size']
    ncols = options['working_size'] # seems to be redundant 
    #D = scipy.misc.imresize(IF, [nrows, ncols], interp='bilinear')
    D = skimage.transform.resize(IF, [nrows, ncols])
    meanD = np.mean(D, axis = 2) # e.q. of mean(D,3) in Matlab
    meanD = np.divide(meanD, np.mean(np.reshape(meanD, (meanD.shape[0]*meanD.shape[1], 1)))) # meanD is a two dim matrix
    W_meanD = dct(meanD)
    
    if options['lambda'] is None: 
        options['lambda'] = np.divide(np.sum(np.abs(W_meanD.flatten('F'))), 400*0.5)
    if options['lambda_darkfield'] is None:
        options['lambda_darkfield'] = np.divide(np.sum(np.abs(W_meanD.flatten('F'))), 400*0.2)
        
    D = np.sort(D,2); # e.q. of sort(D,3) in Matlab
    x,y,z = D.shape
    for i in range(0,z):
        D[:,:,i] = D[:,:,i].transpose(0,1)
    XAoffset = np.zeros([nrows,ncols])
    

    weight = np.ones(D.shape)
    i = 0
    flag_reweighting = True
    flatfield_last = np.ones([nrows,ncols])
    darkfield_last = np.random.randn(nrows,ncols)
    
    
    while (flag_reweighting and i < 3):
        i = i+1
        print('Reweighting Iteration ', i)
        try:
            X_k_A, X_k_E, X_k_Aoffset = inexact_alm_rspca_l1(D, np.ones(IF.shape, dtype=float), 0.001, 2, 1, options)
            XA = np.reshape(X_k_A, [nrows, ncols, IF.shape[2]])
            XE = np.reshape(X_k_E, [nrows, ncols, IF.shape[2]])
            XAoffset = np.reshape(X_k_Aoffset, [nrows, ncols])
            size_XA_z = XA.shape[2]
            mean_XA = np.zeros((1,1,size_XA_z), dtype=float)
            for i in range(0,size_XA_z):
                mean_XA[:,:,i] = np.mean(XA[:,:,i])
            
            temp1 = np.tile(mean_XA, (nrows, ncols, 1)) 
            XE_norm = np.divide(XE, temp1 + 1e-6)
            
            weight = np.divide(1.0, np.add(np.abs(XE_norm), options['epsilon']))
            weight = weight * np.size(weight) / np.sum(weight)
            
            temp = np.mean(XA, 2) - XAoffset
            
            flatfield_current = np.divide(temp, np.mean(temp))
            
            darkfield_current = XAoffset
            mad_flatfield = np.divide(np.sum(np.abs(flatfield_current.flatten('F') - flatfield_last.flatten('F'))), np.sum(np.abs(flatfield_last.flatten('F'))))
            
            temp_diff = np.sum(np.abs(darkfield_current.flatten('F') - darkfield_last.flatten('F')))
            
            if (temp_diff < 1e-7):
                mad_darkfield = 0
            else:
                temp_max = np.maximum(np.sum(np.abs(darkfield_last.flatten('F'))), 1)
                mad_darkfield = temp_diff / temp_max

            
            flatfield_last = flatfield_current
            darkfield_last = darkfield_current
            if np.maximum(mad_flatfield, mad_darkfield) <= options['reweight_tol'] or i > options['max_reweightiterations']:
                flag_reweighting = 0
        except Exception as e:
            exc_type, exc_obj, exc_tb = sys.exc_info()
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            print(exc_type, fname, exc_tb.tb_lineno)
            print(e)
       
    shading =  np.mean(XA,2) - XAoffset
    #XAoffset = XAoffset+B_offset.*shading;
    flatfield = scipy.misc.imresize(shading, [IF.shape[0], IF.shape[1]])
    flatfield = np.divide(flatfield, np.mean(flatfield.flatten('F')))
    if options['darkfield']:
        darkfield = XAoffset
        #darkfield = scipy.misc.imresize(np.reshape(XAoffset,(nrows,ncols,1)),[IF.shape[0], IF.shape[1]])
    else:
        darkfield = np.empty # Seems unnecessary
    
    
    return darkfield